In [ ]:
%pip install langchain >> null
%pip install openai >> null
%pip install langchain_openai >> null
%pip install dotenv >> null

In [2]:
import os
import requests
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

/Users/aleksandrmalysev/Yandex.Disk.localized/Developer/sdexperiment/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
load_dotenv()
# -----------------------------------------------------------------------------
# VeyraX API Helper Function
# -----------------------------------------------------------------------------
# This function is modeled after the call_veyrax_api definition found in the
# Quick Start VeyraX notebook (see :contentReference[oaicite:1]{index=1}). It constructs the URL,
# sets the required header (with VEYRAX_API_KEY), and makes the HTTP request.
VEYRAX_API_KEY = os.environ.get("VEYRAX_API_KEY", "your_api_key_here")
BASE_URL = os.environ.get("VEYRAX_BASE_URL", "https://veyraxapp.com")
openai_api_key = os.environ.get("OPENAI_API_KEY", "your_api_key_here")

def call_veyrax_api(endpoint: str, method: str = "POST", params: dict = None, data: dict = None, json: dict = None) -> dict:
    url = f"{BASE_URL}{endpoint}"
    headers = {"VEYRAX_API_KEY": VEYRAX_API_KEY}
    try:
        response = requests.request(method, url, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()
    except requests.RequestException as e:
        print(f"Error calling API: {e}")
        return {"error": str(e)}

In [11]:
# -----------------------------------------------------------------------------
# Define LangChain Tools for VeyraX Calls
# -----------------------------------------------------------------------------
# Tool 1: Get available VeyraX tools (uses a GET request)
@tool
def get_veyrax_tools() -> dict:
    """Fetch available tools from VeyraX."""
    return call_veyrax_api("/get-tools", method="GET")

# Tool 2: Create a Gmail draft via VeyraX
@tool
def create_gmail_draft(recipient: str, subject: str, body: str) -> dict:
    """Create a Gmail draft using VeyraX."""
    payload = {"draft_args": {"recipient": recipient, "subject": subject, "body": body}}
    return call_veyrax_api("/tool-call/gmail/create_draft", json=payload)

# Tool 3: Send a Gmail message via VeyraX (assumes a draft has been created)
@tool
def send_gmail_message(message_id: str) -> dict:
    """Send a Gmail message using VeyraX."""
    return call_veyrax_api("/tool-call/gmail/send_message", json={"message_id": message_id})

In [12]:
from IPython.display import IFrame

In [13]:
def render_dynamic_components_page(component):
    import json
    # Serialize the components list (each with keys: name, props, jsx)
    components_json = json.dumps(component)
    
    with open("dynamic_components.html", "r", encoding="utf-8") as f:
        template = f.read()
        
    return template.replace("{components_json}", components_json)
def render_and_show(component):
    html = render_dynamic_components_page(component)
    with open("component.html", "w") as f:
        f.write(html)
    return IFrame(src='component.html', width=400, height=200)

In [14]:
# -----------------------------------------------------------------------------
# Bind Tools to an LLM and Invoke Them
# -----------------------------------------------------------------------------
# Initialize the LLM (this example uses ChatOpenAI with 4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

# Bind our VeyraX tools so the model can generate calls to them
llm_with_tools = llm.bind_tools([get_veyrax_tools, create_gmail_draft, send_gmail_message])

In [15]:
# Example: Fetch the list of available VeyraX tools
tools_info = get_veyrax_tools.invoke("Fetch available tools")

In [16]:
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | create_gmail_draft | (lambda x: x.get('component')) | render_and_show

In [17]:
chain.invoke("Create an email with recipient hello@veyrax.com, subject 'Demo', and body 'Hello World!'")

In [19]:
user_tools = get_veyrax_tools.invoke("What are user tools?")

In [20]:
user_tools

{'tools': {'notion': {'search': {'type': 'function',
    'function': {'name': '/notion/search',
     'strict': True,
     'parameters': {'type': 'object',
      'title': 'SearchParams',
      'required': ['query', 'start_cursor', 'page_size', 'sort_direction'],
      'properties': {'query': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
        'title': 'Query',
        'default': ''},
       'page_size': {'anyOf': [{'type': 'integer', 'maximum': 100},
         {'type': 'null'}],
        'title': 'Page Size',
        'default': 100},
       'start_cursor': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
        'title': 'Start Cursor'},
       'sort_direction': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
        'title': 'Sort Direction'}},
      'additionalProperties': False},
     'description': 'Handle POST /notion/search'}},
   'get_page': {'type': 'function',
    'function': {'name': '/notion/get_page',
     'strict': True,
     'parameters': {'type': 'object',
      't